<a href="https://colab.research.google.com/github/lqh-0514/6.894/blob/master/Visual_Encoding_with_Vega_Lite_%26_Altair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visual Encodings with Vega-Lite & Altair

The data and image models we learned about last time gave us a language for talking about visualization design. Today, we'll be more concrete by using those models to construct visualizations.

In particular, we'll be using [Altair](https://altair-viz.github.io/) which provides a Pythonic interface to the [Vega-Lite](https://vega.github.io/vega-lite/) visualization grammar. By "grammar", we mean that the *data*, *transformations*, *marks*, and *encoding channels* we learned about last time are all exposed as building blocks. Creating visualizations involves combining these building blocks, and describing their properties, rather than writing for-loops and low-level drawing commands. This approach is known as *declarative specification*: describing *what* we want the visualization to look like, rather than *how* it should be computed and drawn. 

To learn more about Vega-Lite and declarative specification, watch the [Vega-Lite presentation video from OpenVisConf 2017](https://www.youtube.com/watch?v=9uaHRWj04D4).

This notebook will guide you through the basic process of creating visualizations in Altair. We recommend keeping a reference to the [Altair documentation](https://altair-viz.github.io/) handy!

## Imports

To start, we must import the necessary libraries: Pandas for data frames and Altair for visualization.

In [0]:
import pandas as pd
import altair as alt

## Data

Data in Altair is built around the Pandas data frame, which consists of a set of named data *columns*. We will also regularly refer to data columns as data *fields*.

Altair makes it easy to load example datasets from the [vega-datasets](https://github.com/vega/vega-datasets) collection into a Pandas data frame. In this notebook, we will be visualizing global health and population data for a number of countries, over the time period of 1955 to 2005. The data was collected by the [Gapminder Foundation](https://www.gapminder.org/) and shared in [Hans Rosling's popular TED talk](https://www.youtube.com/watch?v=hVimVzgtD6w). If you haven't seen the talk, we encourage you to watch it!

In [0]:
from vega_datasets import data as vega_data
data = vega_data.gapminder()

How big is the data?

In [0]:
data.shape

(693, 6)

That's 693 rows, and 6 columns! 

We can also take a peak at the data to examine it in more detail:

In [0]:
data.head(5)

,cluster,country,fertility,life_expect,pop,year
0,0,Afghanistan,7.7,30.332,8891209,1955
1,0,Afghanistan,7.7,31.997,9829450,1960
2,0,Afghanistan,7.7,34.020,10997885,1965
3,0,Afghanistan,7.7,36.088,12430623,1970
4,0,Afghanistan,7.7,38.438,14132019,1975


So, for each `country` and `year` (in 5-year intervals), we have measures of fertility in terms of the number of children per woman (`fertility`), life expectancy in years (`life_expect`), and total population (`pop`).

We also see a `cluster` field with an integer code. What might this represent? We'll try and solve this mystery as we visualize the data!

We can also create a smaller data frame, but filtering down the data like so:

In [0]:
data2000 = data.loc[data['year'] == 2000]

In [0]:
data2000.head(5)

,cluster,country,fertility,life_expect,pop,year
9,0,Afghanistan,7.4792,42.129,23898198,2000
20,3,Argentina,2.3500,74.340,37497728,2000
31,3,Aruba,2.1240,73.451,69539,2000
42,4,Australia,1.7560,80.370,19164620,2000
53,1,Austria,1.3820,78.980,8113413,2000


For more information about data frames - and some useful transformations to prepare Pandas data frames for plotting with Altair! - see the [Specifying Data with Altair documentation](https://altair-viz.github.io/user_guide/data.html).

## The Chart Object

The fundamental object in Altair is the `Chart`, which takes a data frame as a single argument:

In [0]:
chart = alt.Chart(data2000)

So far, we have defined the `Chart` object and passed it the simple data frame we generated above. We have not yet told the chart to *do* anything with the data.

## Marks and Encodings

With a chart object in hand, we can now specify how we would like the data to be visualized. We first indicate what kind of graphical *mark* (geometric shape) we want to use to represent the data. We can set the `mark` attribute of the chart object using the the `Chart.mark_*` methods.

For example, we can show the data as a point using `Chart.mark_point()`:

In [0]:
alt.Chart(data2000).mark_point()

Chart({
  data:      cluster             country  fertility  life_expect         pop  year
  9          0         Afghanistan     7.4792       42.129    23898198  2000
  20         3           Argentina     2.3500       74.340    37497728  2000
  31         3               Aruba     2.1240       73.451       69539  2000
  42         4           Australia     1.7560       80.370    19164620  2000
  53         1             Austria     1.3820       78.980     8113413  2000
  64         3             Bahamas     2.1111       71.068      290075  2000
  75         0          Bangladesh     3.2240       62.013   130406594  2000
  86         3            Barbados     1.5000       75.970      273483  2000
  97         1             Belgium     1.6380       78.320    10263618  2000
  108        3             Bolivia     3.9585       63.883     8152620  2000
  119        3              Brazil     2.3450       71.006   175552771  2000
  130        3              Canada     1.5220       79.770    31278097  2000
  141        3               Chile     2.0000       77.860    15153450  2000
  152        4               China     1.7000       72.028  1262645000  2000
  163        3            Colombia     2.4705       71.682    39685655  2000
  174        3          Costa Rica     2.2815       78.123     3710558  2000
  185        1             Croatia     1.3480       74.876     4410830  2000
  196        3                Cuba     1.6300       77.158    11134273  2000
  207        3  Dominican Republic     2.9500       70.847     8385828  2000
  218        3             Ecuador     2.8175       74.173    12505204  2000
  229        5               Egypt     3.1740       69.806    70492342  2000
  240        3         El Salvador     2.8830       70.734     6122515  2000
  251        1             Finland     1.7540       78.370     5168595  2000
  262        1              France     1.8833       79.590    59381628  2000
  273        1             Georgia     1.4780       70.476     4777209  2000
  284        1             Germany     1.3460       78.670    82187909  2000
  295        1              Greece     1.2770       78.256    10559110  2000
  306        3             Grenada     2.4290       67.746       89312  2000
  317        3               Haiti     4.0000       58.137     7306302  2000
  328        4           Hong Kong     0.9400       81.495     6658720  2000
  ..       ...                 ...        ...          ...         ...   ...
  372        5                Iran     2.1240       69.451    65660289  2000
  383        5                Iraq     4.8580       57.046    22675617  2000
  394        1             Ireland     1.9690       77.783     3791690  2000
  405        5              Israel     2.9060       79.696     5842454  2000
  416        1               Italy     1.2860       80.240    57719337  2000
  427        3             Jamaica     2.6280       72.047     2615467  2000
  438        4               Japan     1.2910       82.000   126699784  2000
  449        2               Kenya     5.0000       50.992    29985839  2000
  460        4         South Korea     1.9173       66.662    21647682  2000
  471        4         North Korea     1.2420       77.045    47351083  2000
  482        5             Lebanon     2.3190       71.028     3578036  2000
  493        3              Mexico     2.4005       74.902    99926620  2000
  504        1         Netherlands     1.7260       78.530    15907853  2000
  515        4         New Zealand     1.9640       79.110     3819762  2000
  526        2             Nigeria     5.8450       46.608   114306700  2000
  537        1              Norway     1.8010       79.050     4492400  2000
  548        0            Pakistan     3.9936       63.610   146342958  2000
  559        3                Peru     2.7005       69.906    25979722  2000
  570        4         Philippines     3.5436       70.303    79739825  2000
  581        1              Poland     1.2510       74.670 

Here the rendering consists of one point per row in the dataset, all plotted on top of each other, since we have not yet specified positions for these points.

To visually separate the points, we can map various *encoding channels*, or *channels* for short, to fields in the dataset. For example, we could *encode* the field `fertility` of the data with the `x` channel, which represents the x-axis position of the points. To specify this, use the `Chart.encode` method:

In [0]:
alt.Chart(data2000).mark_point().encode(
  x='fertility',
)

Chart({
  data:      cluster             country  fertility  life_expect         pop  year
  9          0         Afghanistan     7.4792       42.129    23898198  2000
  20         3           Argentina     2.3500       74.340    37497728  2000
  31         3               Aruba     2.1240       73.451       69539  2000
  42         4           Australia     1.7560       80.370    19164620  2000
  53         1             Austria     1.3820       78.980     8113413  2000
  64         3             Bahamas     2.1111       71.068      290075  2000
  75         0          Bangladesh     3.2240       62.013   130406594  2000
  86         3            Barbados     1.5000       75.970      273483  2000
  97         1             Belgium     1.6380       78.320    10263618  2000
  108        3             Bolivia     3.9585       63.883     8152620  2000
  119        3              Brazil     2.3450       71.006   175552771  2000
  130        3              Canada     1.5220       79.770    31278097  2000
  141        3               Chile     2.0000       77.860    15153450  2000
  152        4               China     1.7000       72.028  1262645000  2000
  163        3            Colombia     2.4705       71.682    39685655  2000
  174        3          Costa Rica     2.2815       78.123     3710558  2000
  185        1             Croatia     1.3480       74.876     4410830  2000
  196        3                Cuba     1.6300       77.158    11134273  2000
  207        3  Dominican Republic     2.9500       70.847     8385828  2000
  218        3             Ecuador     2.8175       74.173    12505204  2000
  229        5               Egypt     3.1740       69.806    70492342  2000
  240        3         El Salvador     2.8830       70.734     6122515  2000
  251        1             Finland     1.7540       78.370     5168595  2000
  262        1              France     1.8833       79.590    59381628  2000
  273        1             Georgia     1.4780       70.476     4777209  2000
  284        1             Germany     1.3460       78.670    82187909  2000
  295        1              Greece     1.2770       78.256    10559110  2000
  306        3             Grenada     2.4290       67.746       89312  2000
  317        3               Haiti     4.0000       58.137     7306302  2000
  328        4           Hong Kong     0.9400       81.495     6658720  2000
  ..       ...                 ...        ...          ...         ...   ...
  372        5                Iran     2.1240       69.451    65660289  2000
  383        5                Iraq     4.8580       57.046    22675617  2000
  394        1             Ireland     1.9690       77.783     3791690  2000
  405        5              Israel     2.9060       79.696     5842454  2000
  416        1               Italy     1.2860       80.240    57719337  2000
  427        3             Jamaica     2.6280       72.047     2615467  2000
  438        4               Japan     1.2910       82.000   126699784  2000
  449        2               Kenya     5.0000       50.992    29985839  2000
  460        4         South Korea     1.9173       66.662    21647682  2000
  471        4         North Korea     1.2420       77.045    47351083  2000
  482        5             Lebanon     2.3190       71.028     3578036  2000
  493        3              Mexico     2.4005       74.902    99926620  2000
  504        1         Netherlands     1.7260       78.530    15907853  2000
  515        4         New Zealand     1.9640       79.110     3819762  2000
  526        2             Nigeria     5.8450       46.608   114306700  2000
  537        1              Norway     1.8010       79.050     4492400  2000
  548        0            Pakistan     3.9936       63.610   146342958  2000
  559        3                Peru     2.7005       69.906    25979722  2000
  570        4         Philippines     3.5436       70.303    79739825  2000
  581        1              Poland     1.2510       74.670 

The `encode()` method builds a key-value mapping between encoding channels (such as `x`, `y`, `color`, `shape`, `size`, *etc.*) to fields in the dataset, accessed by field name. 

For Pandas data frames, Altair automatically determines an appropriate data type for the mapped column. In this case, Altair uses the *quantitative* type for numerical values. 

But, we can also explicitly annotate data types -- necessary when we don't use Pandas, or when we want to override the default inference (e.g., treating numeric data as nominal).

For instance, here we've encoded the `cluster` field on the `y` channel. Though it is numeric data, we want to treat clusters as discrete values and so annotate it as an ordinal (`O`) data type. 

**What would have happened if we didn't include this annotation?** Try it out in the cell below!

In [0]:
alt.Chart(data2000).mark_point().encode(
    x='fertility',
    y='cluster:O'
)

Chart({
  data:      cluster             country  fertility  life_expect         pop  year
  9          0         Afghanistan     7.4792       42.129    23898198  2000
  20         3           Argentina     2.3500       74.340    37497728  2000
  31         3               Aruba     2.1240       73.451       69539  2000
  42         4           Australia     1.7560       80.370    19164620  2000
  53         1             Austria     1.3820       78.980     8113413  2000
  64         3             Bahamas     2.1111       71.068      290075  2000
  75         0          Bangladesh     3.2240       62.013   130406594  2000
  86         3            Barbados     1.5000       75.970      273483  2000
  97         1             Belgium     1.6380       78.320    10263618  2000
  108        3             Bolivia     3.9585       63.883     8152620  2000
  119        3              Brazil     2.3450       71.006   175552771  2000
  130        3              Canada     1.5220       79.770    31278097  2000
  141        3               Chile     2.0000       77.860    15153450  2000
  152        4               China     1.7000       72.028  1262645000  2000
  163        3            Colombia     2.4705       71.682    39685655  2000
  174        3          Costa Rica     2.2815       78.123     3710558  2000
  185        1             Croatia     1.3480       74.876     4410830  2000
  196        3                Cuba     1.6300       77.158    11134273  2000
  207        3  Dominican Republic     2.9500       70.847     8385828  2000
  218        3             Ecuador     2.8175       74.173    12505204  2000
  229        5               Egypt     3.1740       69.806    70492342  2000
  240        3         El Salvador     2.8830       70.734     6122515  2000
  251        1             Finland     1.7540       78.370     5168595  2000
  262        1              France     1.8833       79.590    59381628  2000
  273        1             Georgia     1.4780       70.476     4777209  2000
  284        1             Germany     1.3460       78.670    82187909  2000
  295        1              Greece     1.2770       78.256    10559110  2000
  306        3             Grenada     2.4290       67.746       89312  2000
  317        3               Haiti     4.0000       58.137     7306302  2000
  328        4           Hong Kong     0.9400       81.495     6658720  2000
  ..       ...                 ...        ...          ...         ...   ...
  372        5                Iran     2.1240       69.451    65660289  2000
  383        5                Iraq     4.8580       57.046    22675617  2000
  394        1             Ireland     1.9690       77.783     3791690  2000
  405        5              Israel     2.9060       79.696     5842454  2000
  416        1               Italy     1.2860       80.240    57719337  2000
  427        3             Jamaica     2.6280       72.047     2615467  2000
  438        4               Japan     1.2910       82.000   126699784  2000
  449        2               Kenya     5.0000       50.992    29985839  2000
  460        4         South Korea     1.9173       66.662    21647682  2000
  471        4         North Korea     1.2420       77.045    47351083  2000
  482        5             Lebanon     2.3190       71.028     3578036  2000
  493        3              Mexico     2.4005       74.902    99926620  2000
  504        1         Netherlands     1.7260       78.530    15907853  2000
  515        4         New Zealand     1.9640       79.110     3819762  2000
  526        2             Nigeria     5.8450       46.608   114306700  2000
  537        1              Norway     1.8010       79.050     4492400  2000
  548        0            Pakistan     3.9936       63.610   146342958  2000
  559        3                Peru     2.7005       69.906    25979722  2000
  570        4         Philippines     3.5436       70.303    79739825  2000
  581        1              Poland     1.2510       74.670 

## Activity: Practice Visual Encodings

For the following activities, you may find it useful to refer to Altair's documentation on [marks](https://altair-viz.github.io/user_guide/marks.html) and [visual encodings](https://altair-viz.github.io/user_guide/encoding.html).

### 1D or Univariate Visualizations

Univariate visualizations are a useful first step to get a summary of your data values and understand their distribution.

**Recreate the following visualization (called a "strip plot") of life expectancy in the year 2000.**

![A strip plot of life expectancy in the year 2000](https://drive.google.com/uc?export=view&id=17i0ucQrcD5JrtmRPqEy3dl7vSboP0G_1)

Depending on the type of mark used, strip plots may also be known as dot plots. 

Histograms are another type of univariate visualizations, depiciting the number of data values that fall within discrete groups. 

**Recreate the following histogram showing the number of countries in each cluster in the year 2000.** _Hint_: A special `count()` function can be used instead of a field when specifying a visual encoding. 

![Cluster histogram](https://drive.google.com/uc?export=view&id=1Ds66sSGcvoZodJSpEvKd9aBTexurn_RB)



We can also produce histograms of quantitative fields by discretizing the data into _bins_. 

_Note:_ To produce this histogram, we'll need to use the more verbose form of defining encodings (e.g., `alt.X()` instead of `x=`) which allows us to specifying [a binning strategy](https://altair-viz.github.io/user_guide/encoding.html#binning-and-aggregation).

**Recreate the following binned histogram of life expectancy in the year 2000:**

![Binned histogram of life expectancy](https://drive.google.com/uc?export=view&id=1dv1llnCxgR8opOAFooEUiLo0z_bRJBPh)

**Compare and contrast the two univariate summaries of life expectancy.**

What are the strengths of the strip plot vs. the binned histogram and vice-versa?

**Create 2 more univariate visualizations of the full dataset (`data`).** 

Experiment with a variety of mark types and encoding channels.

### 2D or Bivariate Visualizations

While univariate visualizations help us understand one single variable at a time, bivariate (or 2D) visualizations can help us identify the relationship between two variables in our dataset.

**Recreate the following bivariate visualizations (using the full dataset, `data`):**

![A scatterplot of life expectancy against fertility](https://drive.google.com/uc?export=view&id=11AINUEhqdei4Uz19j9qaDtOOeUFfsWUV)

Bivariate visualizations do not have to only use the `x` and `y` encoding channels. For example, **recreate the following bivariate chart:**

![A bubble chart of population against fertility](https://drive.google.com/uc?export=view&id=1Qmdix71Y6rzOxqTHcFlv0GJu3bmO7HiG)

And we can also explore non-quantitative fields as part of bivariate visualizations. For instance, **recreate** China's fertility over the years:

![A line chart of China's fertility](https://drive.google.com/uc?export=view&id=1UzX1glTA71N1lAQYdkJeN1X089y9V-Ku)



Similarly, **recreate the following dot plot of life expectancy over the years:**

![A dot plot of year against life expectancy](https://drive.google.com/uc?export=view&id=1I42Ns11TCKGV_E_A1iWWvMESqlYb-Azj)

And  **[layer](https://altair-viz.github.io/user_guide/compound_charts.html#layer-chart) a reference mean line**:

![Dot plot of life expectancy over years with a reference mean line](https://drive.google.com/uc?export=view&id=1GiJ8mmjXBfTAKiSM4Z0t3KWbuXji8ent)



We can also represent the above data distribution as a **two-dimensional histogram**:

![2D histogram of year x life expectancy](https://drive.google.com/uc?export=view&id=1DE98kz8_5goPMGLsmHv2inIKztg2LPKt)

Or, alternatively, as a **heatmap**:

![Heatmap of year x life expectancy](https://drive.google.com/uc?export=view&id=1LiabjR1aM6z8zVZ4z4KvmnALcGAAAZn0)

**Compare the *size* and *color*-based 2D histograms above. Which encoding do you think should be preferred? Why? In which plot can you more precisely compare the magnitude of individual values? In which plot can you more accurately see the overall density of values?**

**Create 2 more bivariate visualizations, exploring a range of marks and encoding channels.**

### N-D or Multivariate Visualizations



But, of course, we don't have to stop at two dimensions. 

**Recreate the following multi-line chart**. _Hint_: You might find the [`query`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) pandas function handy.

![Multi-line chart of China, India, and the United States' fertility over the years](https://drive.google.com/uc?export=view&id=1V6WS1TN2LdfafLgNZSXCE0AAzVWqZpUT)

Bubble charts are a popular means of visualizing 3+ dimensions. **Recreate the following bubble chart for the year 2000:**

![Gapminder bubble chart](https://drive.google.com/uc?export=view&id=1koeBxLHMZ-VTErxK2ndTDtBnhrQmWa6m)

So far, we've focused on displaying all variables within a single plot. However, we quickly approach the limits of this strategy. For example, if we encode any more data in the bubble chart above (e.g., using opacity or shape), we begin to make it more difficult for readers to decode insights from the visualization

For multi-dimensional data, a promising approach is a family of techniques called _small multiple displays_: using a series of charts that use the same (or similar) visual encodings such that they can be easily compared.

One example of small multiples is the **Trellis plot**: the full dataset is partitioned by a nominal or ordinal variable, and each resulting subset of the data is displayed arrayed across [rows or columns](https://altair-viz.github.io/user_guide/encoding.html?highlight=facet#encoding-channels). These partitions are often called **[facets](https://altair-viz.github.io/user_guide/compound_charts.html#faceted-charts).**

**Recreate the following Trellis plot**, reusing our bubble chart visualization and partitioning our data by `year`.

![Trellis Bubble Chart](https://drive.google.com/uc?export=view&id=1uTAg17wsa2kLGyJNjfQ52_Dq9dDvEx5b)

_Hint: You may find it useful to specify a `width` and `height` in your `Chart` constructor to facilitate comparisons across subviews._

Another example of a small multiples display is called the **Scatterplot Matrix or SPLOM.** Instead of partitioning our data into subsets, the SPLOM [repeats](https://altair-viz.github.io/user_guide/compound_charts.html#repeated-charts) the full dataset and visualizes pairs of variables at a time.  

In [0]:
alt.Chart(data, width=175, height=175).mark_point().encode(
  alt.X(alt.repeat('column'), type='quantitative'),
  alt.Y(alt.repeat('row'), type='quantitative')
).repeat(
  row=['life_expect', 'fertility', 'pop'],
  column=['life_expect', 'fertility', 'pop']
)

RepeatChart({
  repeat: Repeat({
    column: ['life_expect', 'fertility', 'pop'],
    row: ['life_expect', 'fertility', 'pop']
  }),
  spec: Chart({
    data:      cluster         country  fertility  life_expect        pop  year
    0          0     Afghanistan     7.7000       30.332    8891209  1955
    1          0     Afghanistan     7.7000       31.997    9829450  1960
    2          0     Afghanistan     7.7000       34.020   10997885  1965
    3          0     Afghanistan     7.7000       36.088   12430623  1970
    4          0     Afghanistan     7.7000       38.438   14132019  1975
    5          0     Afghanistan     7.8000       39.854   15112149  1980
    6          0     Afghanistan     7.9000       40.822   13796928  1985
    7          0     Afghanistan     8.0000       41.674   14669339  1990
    8          0     Afghanistan     8.0000       41.763   20881480  1995
    9          0     Afghanistan     7.4792       42.129   23898198  2000
    10         0     Afghanistan     7.0685       43.828   29928987  2005
    11         3       Argentina     3.1265       64.399   18927821  1955
    12         3       Argentina     3.0895       65.142   20616009  1960
    13         3       Argentina     3.0490       65.634   22283100  1965
    14         3       Argentina     3.1455       67.065   23962313  1970
    15         3       Argentina     3.4400       68.481   26081880  1975
    16         3       Argentina     3.1500       69.942   28369799  1980
    17         3       Argentina     3.0530       70.774   30675059  1985
    18         3       Argentina     2.9000       71.868   33022202  1990
    19         3       Argentina     2.6300       73.275   35311049  1995
    20         3       Argentina     2.3500       74.340   37497728  2000
    21         3       Argentina     2.2540       75.320   39537943  2005
    22         3           Aruba     5.1500       64.381      53865  1955
    23         3           Aruba     4.3990       66.606      57203  1960
    24         3           Aruba     3.3010       68.336      59020  1965
    25         3           Aruba     2.6510       70.941      59039  1970
    26         3           Aruba     2.4500       71.830      59390  1975
    27         3           Aruba     2.3580       74.116      60266  1980
    28         3           Aruba     2.3000       74.494      64129  1985
    29         3           Aruba     2.2800       74.108      66653  1990
    ..       ...             ...        ...          ...        ...   ...
    663        1  United Kingdom     2.0400       72.010   55632000  1970
    664        1  United Kingdom     1.7200       72.760   56215000  1975
    665        1  United Kingdom     1.8000       74.040   56314000  1980
    666        1  United Kingdom     1.8100       75.007   56620240  1985
    667        1  United Kingdom     1.7800       76.420   57493307  1990
    668        1  United Kingdom     1.7000       77.218   58426014  1995
    669        1  United Kingdom     1.6950       78.471   59522468  2000
    670        1  United Kingdom     1.8150       79.425   60441457  2005
    671        3   United States     3.7060       69.490  165931000  1955
    672        3   United States     3.3140       70.210  180671000  1960
    673        3   United States     2.5450       70.760  194303000  1965
    674        3   United States     2.0160       71.340  205052000  1970
    675        3   United States     1.7880       73.380  215973000  1975
    676        3   United States     1.8250       74.650  227726463  1980
    677        3   United States     1.9240       75.020  238466283  1985
    678        3   United States     2.0250       76.090  250131894  1990
    679        3   United States     1.9940       76.810  266557091  1995
    680        3   United States     2.0380       77.310  282338631  2000
    681        3   United States     2.0540       78.242  295734134  2005
    682        3       Venezuela     6.4585       57.907    6170497  19

**Create 2 more multivariate visualizations, exploring a range of marks and encoding channels.**

## Effective Visual Encoding

When we map data values to encoding channels, Altair/Vega-Lite makes a number of design decisions for us including what colors and sizes are used for plotted points, the size of the visualization, and how the axes ticks are drawn.

To more effectively encode our data, we may wish to customize these defaults. And, in order to do so, we need to dig into _how_ visual encoding occurs. The workhorse that actually performs this mapping is called a _scale_: a function that takes a data value as input (the scale _domain_) and returns a visual value, such as a pixel position or RGB color, as output (the scale _range_). These scale functions are themselves visualized using _guides_: _axes_ visualize scales with spatial ranges and _legends_ visual scales with color, size, or shape ranges.

Let's look again at our bubble chart for the year 2000:

![Gapminder bubble chart](https://drive.google.com/uc?export=view&id=1koeBxLHMZ-VTErxK2ndTDtBnhrQmWa6m)

By default, for linear quantitative scales, Altair/Vega-Lite includes zero to ensure a proper baseline for comparing ratio-valued data. However, in some cases, a zero baseline may be meaningless, or you may want to focus on interval comparisons.

**Use [`alt.Scale`](https://altair-viz.github.io/user_guide/generated/core/altair.Scale.html#altair.Scale) to disable automatic zero-inclusion** for both axes to produce the following visualization:

![Non-zero Gapminder bubble chart](https://drive.google.com/uc?export=view&id=1vsxVpeJoDpOnqhcCcCYIt48_9u4Bn53s)



Now the axis scales no longer include zero by default. Some padding still remains as the axis domain end points are automatically snapped to _nice_ numbers like multiples of 5 or 10. 

**What happens if you disable niceness?**

Next, let's take a look at our _size_ channel. It encodes population but the plotted sizes are dominated by two countries: India and China (add a `tooltip` encoding to confirm this).

One fix might be to have population values map to a wider _range_ of circle sizes. We can specify this by **customizing our size scale's `range` (e.g., `[0, 1000]`)**.

![Bubble Chart with larger bubbles](https://drive.google.com/uc?export=view&id=1lxYc4k1zXd2B0MgQgQRIZYtrTEaxiaDc)


Let's dive deeper into the distibution of populations in the year 2000. Here's a dot plot with tooltips enabled to help us identify which points correspond to which country:

In [0]:
alt.Chart(data2000).mark_circle().encode(
  alt.X('pop'),
  alt.Y('cluster:N'),
  alt.Color('cluster:N'),
  alt.Tooltip('country')
)

Chart({
  data:      cluster             country  fertility  life_expect         pop  year
  9          0         Afghanistan     7.4792       42.129    23898198  2000
  20         3           Argentina     2.3500       74.340    37497728  2000
  31         3               Aruba     2.1240       73.451       69539  2000
  42         4           Australia     1.7560       80.370    19164620  2000
  53         1             Austria     1.3820       78.980     8113413  2000
  64         3             Bahamas     2.1111       71.068      290075  2000
  75         0          Bangladesh     3.2240       62.013   130406594  2000
  86         3            Barbados     1.5000       75.970      273483  2000
  97         1             Belgium     1.6380       78.320    10263618  2000
  108        3             Bolivia     3.9585       63.883     8152620  2000
  119        3              Brazil     2.3450       71.006   175552771  2000
  130        3              Canada     1.5220       79.770    31278097  2000
  141        3               Chile     2.0000       77.860    15153450  2000
  152        4               China     1.7000       72.028  1262645000  2000
  163        3            Colombia     2.4705       71.682    39685655  2000
  174        3          Costa Rica     2.2815       78.123     3710558  2000
  185        1             Croatia     1.3480       74.876     4410830  2000
  196        3                Cuba     1.6300       77.158    11134273  2000
  207        3  Dominican Republic     2.9500       70.847     8385828  2000
  218        3             Ecuador     2.8175       74.173    12505204  2000
  229        5               Egypt     3.1740       69.806    70492342  2000
  240        3         El Salvador     2.8830       70.734     6122515  2000
  251        1             Finland     1.7540       78.370     5168595  2000
  262        1              France     1.8833       79.590    59381628  2000
  273        1             Georgia     1.4780       70.476     4777209  2000
  284        1             Germany     1.3460       78.670    82187909  2000
  295        1              Greece     1.2770       78.256    10559110  2000
  306        3             Grenada     2.4290       67.746       89312  2000
  317        3               Haiti     4.0000       58.137     7306302  2000
  328        4           Hong Kong     0.9400       81.495     6658720  2000
  ..       ...                 ...        ...          ...         ...   ...
  372        5                Iran     2.1240       69.451    65660289  2000
  383        5                Iraq     4.8580       57.046    22675617  2000
  394        1             Ireland     1.9690       77.783     3791690  2000
  405        5              Israel     2.9060       79.696     5842454  2000
  416        1               Italy     1.2860       80.240    57719337  2000
  427        3             Jamaica     2.6280       72.047     2615467  2000
  438        4               Japan     1.2910       82.000   126699784  2000
  449        2               Kenya     5.0000       50.992    29985839  2000
  460        4         South Korea     1.9173       66.662    21647682  2000
  471        4         North Korea     1.2420       77.045    47351083  2000
  482        5             Lebanon     2.3190       71.028     3578036  2000
  493        3              Mexico     2.4005       74.902    99926620  2000
  504        1         Netherlands     1.7260       78.530    15907853  2000
  515        4         New Zealand     1.9640       79.110     3819762  2000
  526        2             Nigeria     5.8450       46.608   114306700  2000
  537        1              Norway     1.8010       79.050     4492400  2000
  548        0            Pakistan     3.9936       63.610   146342958  2000
  559        3                Peru     2.7005       69.906    25979722  2000
  570        4         Philippines     3.5436       70.303    79739825  2000
  581        1              Poland     1.2510       74.670 

This dot plot isn't terrible informative right now because we have two outliers: the populations of India and China are orders of magnitude larger than other countries. 

So what are our options? 

One might be to just **adjust the domain of our x-scale** and **[clip the outliers](https://altair-viz.github.io/user_guide/customization.html?highlight=clip#adjusting-axis-limits)**:

![Clipped population dot plot](https://drive.google.com/uc?export=view&id=1em5qy126q1T_hfzZLupOCVAn0Pnr17eC)



**What are the pros and cons of this strategy?**

An alternate strategy might preserve the outliers but better depict that population values span several orders of magnitude. 

By default, Altair/Vega-Lite uses a `linear` mapping (`y = mx + b`) between the domain values (`population`) and the range values (_pixels_). To get a better overview of this data, we may  instead favor a **logarithmic (`log`) scale**:

![Log dot plot of population in 2000](https://drive.google.com/uc?export=view&id=1-sRZg_d6hf5Vesx4C9joBWOn3OFLK4X9)


Now, the domain values (`population`) map to range values (_pixels_) via the function `y = mlog(x) + b`.

And, if we were only interested in demarcating every order of magnitude, we could **tweak [the number of x-axis ticks](https://altair-viz.github.io/user_guide/generated/core/altair.Axis.html?highlight=ticks)** for a less noisy display:

![Log dot plot of population with 5 x-axis ticks](https://drive.google.com/uc?export=view&id=1UkK5CcOJVvtQbfVM_bF2VP-RrMpwTjES)


Besides addressing data skew, log scales are also useful when we want to focus on **multiplicative factors**. In a standard linear scale, a visual (pixel) distance of 10 units might correspond to an *addition* of 10 units in the data domain. A logarithmic transform maps between multiplication and addition, such that `log(u) + log(v) = log(u*v)`. As a result, in a logarithmic scale, a visual distance of 10 units instead corresponds to *multiplication* by 10 units in the data domain (assuming a base 10 logarithm).

For instance, if we want to visualize Google's stock price over time, the default (`linear`) scale allows us to see the absolute change:


In [0]:
stocks = vega_data.stocks()
goog = stocks.loc[stocks['symbol'] == 'GOOG']
alt.Chart(goog).mark_line().encode(
  alt.X('date'),
  alt.Y('price')
)

Chart({
  data:     symbol       date   price
  369   GOOG 2004-08-01  102.37
  370   GOOG 2004-09-01  129.60
  371   GOOG 2004-10-01  190.64
  372   GOOG 2004-11-01  181.98
  373   GOOG 2004-12-01  192.79
  374   GOOG 2005-01-01  195.62
  375   GOOG 2005-02-01  187.99
  376   GOOG 2005-03-01  180.51
  377   GOOG 2005-04-01  220.00
  378   GOOG 2005-05-01  277.27
  379   GOOG 2005-06-01  294.15
  380   GOOG 2005-07-01  287.76
  381   GOOG 2005-08-01  286.00
  382   GOOG 2005-09-01  316.46
  383   GOOG 2005-10-01  372.14
  384   GOOG 2005-11-01  404.91
  385   GOOG 2005-12-01  414.86
  386   GOOG 2006-01-01  432.66
  387   GOOG 2006-02-01  362.62
  388   GOOG 2006-03-01  390.00
  389   GOOG 2006-04-01  417.94
  390   GOOG 2006-05-01  371.82
  391   GOOG 2006-06-01  419.33
  392   GOOG 2006-07-01  386.60
  393   GOOG 2006-08-01  378.53
  394   GOOG 2006-09-01  401.90
  395   GOOG 2006-10-01  476.39
  396   GOOG 2006-11-01  484.81
  397   GOOG 2006-12-01  460.48
  398   GOOG 2007-01-01  501.50
  ..     ...        ...     ...
  407   GOOG 2007-10-01  707.00
  408   GOOG 2007-11-01  693.00
  409   GOOG 2007-12-01  691.48
  410   GOOG 2008-01-01  564.30
  411   GOOG 2008-02-01  471.18
  412   GOOG 2008-03-01  440.47
  413   GOOG 2008-04-01  574.29
  414   GOOG 2008-05-01  585.80
  415   GOOG 2008-06-01  526.42
  416   GOOG 2008-07-01  473.75
  417   GOOG 2008-08-01  463.29
  418   GOOG 2008-09-01  400.52
  419   GOOG 2008-10-01  359.36
  420   GOOG 2008-11-01  292.96
  421   GOOG 2008-12-01  307.65
  422   GOOG 2009-01-01  338.53
  423   GOOG 2009-02-01  337.99
  424   GOOG 2009-03-01  348.06
  425   GOOG 2009-04-01  395.97
  426   GOOG 2009-05-01  417.23
  427   GOOG 2009-06-01  421.59
  428   GOOG 2009-07-01  443.05
  429   GOOG 2009-08-01  461.67
  430   GOOG 2009-09-01  495.85
  431   GOOG 2009-10-01  536.12
  432   GOOG 2009-11-01  583.00
  433   GOOG 2009-12-01  619.98
  434   GOOG 2010-01-01  529.94
  435   GOOG 2010-02-01  526.80
  436   GOOG 2010-03-01  560.19
  
  [68 rows x 3 columns],
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'date'
    }),
    y: Y({
      shorthand: 'price'
    })
  }),
  mark: 'line'
})

By changing `price` to use a `log` scale, we instead visualize the percentage change. Each unit of distance now reflects an equal percentage change (i.e., percentage gains take up the same amount of space). For example, the vertical distance between \$100 and \$200 and \$300 and \$600 is the same as each range represents a 100% increase.

In [0]:
alt.Chart(goog).mark_line().encode(
  alt.X('date'),
  alt.Y('price', scale=alt.Scale(type='log'))
)

Chart({
  data:     symbol       date   price
  369   GOOG 2004-08-01  102.37
  370   GOOG 2004-09-01  129.60
  371   GOOG 2004-10-01  190.64
  372   GOOG 2004-11-01  181.98
  373   GOOG 2004-12-01  192.79
  374   GOOG 2005-01-01  195.62
  375   GOOG 2005-02-01  187.99
  376   GOOG 2005-03-01  180.51
  377   GOOG 2005-04-01  220.00
  378   GOOG 2005-05-01  277.27
  379   GOOG 2005-06-01  294.15
  380   GOOG 2005-07-01  287.76
  381   GOOG 2005-08-01  286.00
  382   GOOG 2005-09-01  316.46
  383   GOOG 2005-10-01  372.14
  384   GOOG 2005-11-01  404.91
  385   GOOG 2005-12-01  414.86
  386   GOOG 2006-01-01  432.66
  387   GOOG 2006-02-01  362.62
  388   GOOG 2006-03-01  390.00
  389   GOOG 2006-04-01  417.94
  390   GOOG 2006-05-01  371.82
  391   GOOG 2006-06-01  419.33
  392   GOOG 2006-07-01  386.60
  393   GOOG 2006-08-01  378.53
  394   GOOG 2006-09-01  401.90
  395   GOOG 2006-10-01  476.39
  396   GOOG 2006-11-01  484.81
  397   GOOG 2006-12-01  460.48
  398   GOOG 2007-01-01  501.50
  ..     ...        ...     ...
  407   GOOG 2007-10-01  707.00
  408   GOOG 2007-11-01  693.00
  409   GOOG 2007-12-01  691.48
  410   GOOG 2008-01-01  564.30
  411   GOOG 2008-02-01  471.18
  412   GOOG 2008-03-01  440.47
  413   GOOG 2008-04-01  574.29
  414   GOOG 2008-05-01  585.80
  415   GOOG 2008-06-01  526.42
  416   GOOG 2008-07-01  473.75
  417   GOOG 2008-08-01  463.29
  418   GOOG 2008-09-01  400.52
  419   GOOG 2008-10-01  359.36
  420   GOOG 2008-11-01  292.96
  421   GOOG 2008-12-01  307.65
  422   GOOG 2009-01-01  338.53
  423   GOOG 2009-02-01  337.99
  424   GOOG 2009-03-01  348.06
  425   GOOG 2009-04-01  395.97
  426   GOOG 2009-05-01  417.23
  427   GOOG 2009-06-01  421.59
  428   GOOG 2009-07-01  443.05
  429   GOOG 2009-08-01  461.67
  430   GOOG 2009-09-01  495.85
  431   GOOG 2009-10-01  536.12
  432   GOOG 2009-11-01  583.00
  433   GOOG 2009-12-01  619.98
  434   GOOG 2010-01-01  529.94
  435   GOOG 2010-02-01  526.80
  436   GOOG 2010-03-01  560.19
  
  [68 rows x 3 columns],
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'date'
    }),
    y: Y({
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'price'
    })
  }),
  mark: 'line'
})

Log scales, however, do come with some constraints:

*   Only positive numbers are supported as the logarithm of negative numbers is undefined.
*   They require some level of audience familiarity -- your readers must understand that log scales depict changes in terms of orders of magnitude or percentage change.



## Submit

Remember to submit your notebook to LMOD by doing the following:

1. Click the _Share_ button in the upper right-hand corner.
2. Click _Get a Shareable Link_ in the dialog box, and double check that _Anyone with a link **can view**_ is selected.

## Acknowledgements

This notebook was adapted from material developed by Jeffrey Heer, Dominik Moritz, and Brock Craft.